In [2]:
from deep_code.tools.publish import Publisher

# remember -> .gitaccess and .env
# https://eu-west-2.console.aws.amazon.com/s3/buckets/pangeo-test-fires/object/delete?region=eu-west-2&bucketType=general&showversions=false

ModuleNotFoundError: No module named 'deep_code'

In [2]:
import os

os.chdir("/home/lampata/Demo/deep-code")

def load_env_file(filepath=".env"):
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#'):
                continue 
            if '=' in line:
                key, value = line.split('=', 1)
                key = key.strip()
                value = value.strip().strip('"').strip("'") 
                os.environ[key] = value

load_env_file()

In [3]:

os.chdir("/home/lampata/Demo/")

In [4]:
import xarray as xr
from xcube.core.store import new_data_store

# store data on s3
root="pangeo-test-fires"

# Path to the local Zarr dataset
zarr_path = "wildfires-foss4g/dnbr_dataset.zarr"

# Open the Zarr dataset
ds = xr.open_zarr(zarr_path)

ds

store = new_data_store(
    "s3",
    root=root,
    storage_options={
                "anon": False,
                "key": os.environ.get("S3_USER_STORAGE_KEY"),
                "secret": os.environ.get("S3_USER_STORAGE_SECRET"),
                "client_kwargs": {
                "endpoint_url": "https://s3.eu-west-2.amazonaws.com",
                "region_name": os.environ.get("AWS_DEFAULT_REGION")
            }
            },
    
)
store.write_data(ds, "dnbr_dataset.zarr", replace=True)

'dnbr_dataset.zarr'

In [5]:
os.chdir("/home/lampata/Demo/deep-code")

In [8]:
dataset_config="""
dataset_id: dnbr_dataset.zarr
collection_id: pangeo-test
osc_themes:
- 'land'
documentation_link: https://www.sciencedirect.com/science/article/pii/S1470160X22004708#f0035
access_link: s3://pangeo-test-fires
dataset_status: completed
osc_region: global
cf_parameter:
  - name: burned-area
"""

In [9]:
workflow_config="""
workflow_id: "dnbr_workflow_example"
properties:
  title: "DNBR Workflow Example"
  description: "Demonstrate how to fetch satellite Sentinel-2 data to generate burn severity maps for the assessment of the areas affected by wildfires."
  keywords:
    - Earth Science
  themes:
      - land
  license: proprietary
  jupyter_kernel_info:
    name: Pange-Test-Notebook
    python_version: 3.11
    env_file: "https://github.com/pangeo-data/pangeo-docker-images/blob/master/pangeo-notebook/environment.yml"
jupyter_notebook_url: "https://github.com/pangeo-data/pangeo-openeo-BiDS-2023/blob/main/tutorial/examples/dask/wildfires_daskgateway.ipynb"
contact:
  - name: Dean Summers
    organization: Lampata
    links:
      - rel: "about"
        type: "text/html"
        href: "https://www.lampata.eu/"
"""

In [10]:
with open("workflow_config.yaml", 'w') as f:
    f.write(workflow_config)

In [11]:
with open("dataset_config.yaml", 'w') as f:
    f.write(dataset_config)

In [12]:
# !git config --global user.email "dean@lampata.co.uk"
# !git config --global user.name "Dean S"


In [13]:

publisher = Publisher(
    dataset_config_path="dataset_config.yaml",
    workflow_config_path="workflow_config.yaml",
    environment="staging",
)
publisher.publish_all()


INFO:root:Forking repository...
INFO:root:Repository forked to sunnydean/open-science-catalog-metadata-staging
INFO:root:Checking local repository...
INFO:root:Cloning forked repository...
Cloning into '/home/lampata/temp_repo'...
Updating files: 100% (1776/1776), done.
INFO:root:Repository cloned to /home/lampata/temp_repo
INFO:deep_code.tools.publish:Generating STAC collection...
INFO:deep_code.utils.dataset_stac_generator:Attempting to open dataset with configuration: Public store
ERROR:deep_code.utils.dataset_stac_generator:Failed to open dataset with configuration: Public store. Error: Failed to open dataset 'deep-esdl-public/dnbr_dataset.zarr': group not found at path ''
INFO:deep_code.utils.dataset_stac_generator:Attempting to open dataset with configuration: Authenticated store
INFO:deep_code.utils.dataset_stac_generator:Successfully opened dataset with configuration: Authenticated store
INFO:deep_code.tools.publish:Variable catalog for delta-nbr does not exist. Creating...


Enter GCMD keyword URL or a similar url for delta-nbr:   https://gcmd.earthdata.nasa.gov/KeywordViewer/scheme/Earth%20Science/436b098d-e4d9-4fbd-9ede-05675e111eee?gtm_keyword=BURNED%20AREA&gtm_scheme=Earth%20Science


INFO:deep_code.utils.dataset_stac_generator:Added GCMD link for delta-nbr catalog https://gcmd.earthdata.nasa.gov/KeywordViewer/scheme/Earth%20Science/436b098d-e4d9-4fbd-9ede-05675e111eee?gtm_keyword=BURNED%20AREA&gtm_scheme=Earth%20Science.
INFO:deep_code.tools.publish:Generating OGC API Record for the workflow...
INFO:root:Creating new branch: add-new-collection-pangeo-test-20250508104059...
Switched to a new branch 'add-new-collection-pangeo-test-20250508104059'
INFO:deep_code.tools.publish:Adding products/pangeo-test/collection.json to add-new-collection-pangeo-test-20250508104059
INFO:root:Adding new file: products/pangeo-test/collection.json...
INFO:deep_code.tools.publish:Adding variables/delta-nbr/catalog.json to add-new-collection-pangeo-test-20250508104059
INFO:root:Adding new file: variables/delta-nbr/catalog.json...
INFO:deep_code.tools.publish:Adding /home/lampata/temp_repo/variables/catalog.json to add-new-collection-pangeo-test-20250508104059
INFO:root:Adding new file: /

[add-new-collection-pangeo-test-20250508104059 614e2d9f] Add new dataset collection: pangeo-test and workflow/experiment: dnbr_workflow_example
 9 files changed, 420 insertions(+), 8 deletions(-)
 create mode 100644 experiments/dnbr_workflow_example/record.json
 create mode 100644 products/pangeo-test/collection.json
 create mode 100644 variables/delta-nbr/catalog.json
 create mode 100644 workflows/dnbr_workflow_example/record.json


remote: 
remote: Create a pull request for 'add-new-collection-pangeo-test-20250508104059' on GitHub by visiting:        
remote:      https://github.com/sunnydean/open-science-catalog-metadata-staging/pull/new/add-new-collection-pangeo-test-20250508104059        
remote: 
To https://github.com/sunnydean/open-science-catalog-metadata-staging.git
 * [new branch]        add-new-collection-pangeo-test-20250508104059 -> add-new-collection-pangeo-test-20250508104059
INFO:root:Creating a pull request...


branch 'add-new-collection-pangeo-test-20250508104059' set up to track 'origin/add-new-collection-pangeo-test-20250508104059'.


INFO:root:Pull request created: https://github.com/ESA-EarthCODE/open-science-catalog-metadata-staging/pull/90
INFO:deep_code.tools.publish:Pull request created: None
INFO:root:Cleaning up local repository...
INFO:deep_code.tools.publish:Pull request created: None


In [36]:
import os

def delete_directory_recursive(path):
    if not os.path.exists(path):
        print(f"{path} does not exist.")
        return

    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            file_path = os.path.join(root, name)
            os.remove(file_path)
        for name in dirs:
            os.rmdir(os.path.join(root, name))
    os.rmdir(path)
    print(f"{path} has been deleted.")

# delete_directory_recursive("/home/lampata/temp_repo")
# delete_directory_recursive("/home/lampata/Demo/wildfires-foss4g")

/home/lampata/temp_repo has been deleted.


In [ ]:
# https://gcmd.earthdata.nasa.gov/KeywordViewer/scheme/Earth%20Science/436b098d-e4d9-4fbd-9ede-05675e111eee?gtm_keyword=BURNED%20AREA&gtm_scheme=Earth%20Science